# Cross-validation

Cross-validation is a kind of resampling approach that can be used to estimate the test error associated with a given statistical learning method in order to evaluate its performance, or to select the appropriate level of flexibility. The process of evaluating a model’s performance is known as _model assessment_, whereas the process of selecting the proper level of flexibility for a model is known as _model selection_.

Watch the 6-minute video below for a visual explanation of cross-validation:

```{admonition} Video
<iframe width="700" height="394" src="https://www.youtube.com/embed/fSytzGwwBVw?start=15" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

[Explaining Cross Validation, by StatQuest](https://www.youtube.com/embed/fSytzGwwBVw?start=15)
```

## Import libraries and load data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (
    train_test_split,
    LeaveOneOut,
    KFold,
    cross_val_score,
)
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import resample

%matplotlib inline

We will illustrate cross-validation on the `Auto` dataset. Run the following cell to load this dataset.

In [ ]:
auto_url = "https://github.com/pykale/transparentML/raw/main/data/Auto.csv"

auto_df = pd.read_csv(auto_url, na_values="?").dropna()
auto_df.info()

In [ ]:
auto_df.head()

## Validation Set Approach

<!-- Using [Polynomial](http://scikit-learn.org/dev/modules/preprocessing.html#generating-polynomial-features) feature generation in scikit-learn -->
The validation set approach is a strategy to estimate the _test error_ associated with fitting a particular model on a set of observations. It involves randomly dividing the available set of observations into two parts, a _training set_ and a _validation set_ or _hold-out set_. The model is fit on the training set, and the fitted model is used to predict the responses for the unseen observations in the validation set. The resulting validation set error rate—typically assessed using MSE in the case of a quantitative response—provides an estimate of the test error rate.

Run the following to illustrate the validation set approach on the `Auto` dataset over different polynomial degrees.

In [ ]:
t_prop = 0.5
p_order = np.arange(1, 11)
r_state = np.arange(0, 10)

X, Y = np.meshgrid(p_order, r_state, indexing="ij")
Z = np.zeros((p_order.size, r_state.size))

regr = LinearRegression()

# Generate 10 random splits of the dataset
for (i, j), v in np.ndenumerate(Z):
    poly = PolynomialFeatures(int(X[i, j]))
    X_poly = poly.fit_transform(auto_df.horsepower.values.reshape(-1, 1))

    X_train, X_test, y_train, y_test = train_test_split(
        X_poly, auto_df.mpg.ravel(), test_size=t_prop, random_state=Y[i, j]
    )

    regr.fit(X_train, y_train)
    pred = regr.predict(X_test)
    Z[i, j] = mean_squared_error(y_test, pred)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Left plot (first split)
ax1.plot(X.T[0], Z.T[0], "-o")
ax1.set_title("Random split of the data set")

# Right plot (all splits)
ax2.plot(X, Z)
ax2.set_title("10 random splits of the data set")

for ax in fig.axes:
    ax.set_ylabel("Mean Squared Error")
    ax.set_ylim(15, 30)
    ax.set_xlabel("Degree of Polynomial")
    ax.set_xlim(0.5, 10.5)
    ax.set_xticks(range(2, 11, 2));

In [Linear regression](https://pykale.github.io/transparentML/02-linear-reg/extension-limitation.html#non-linear-relationships), we discovered a non-linear relationship between `mpg` and `horsepower`. Compared to using only a liner term, a model that using `horsepower` and `horsepower`$^2$ gives better results in predicts `mpg`. For the `Auto` dataset, we can randomly split the 392 observations into two sets, a training set containing 196 of the data points for model training, and a validation set containing the remaining 196 observations for evaluating by MSE. As shown in the left-hand panel of the figure above.

If we repeat the process of randomly splitting, we will get a somewhat different estimate for the test MSE. Ten different validation set MSE curves are shown in the right-hand panel of the figure above.
<!-- As an illustration, the right-hand panel of Figure 5.2 displays ten different validation set MSE curves from the Auto data set, produced using ten different random splits of the observations into training and validation sets.  -->
We can observe:
- model with a quadratic term has a dramatically smaller validation set MSE than the model with only a linear term
- ot much benefit in including cubic or higher-order polynomial terms in the model
- each of the ten curves results in a different test MSE estimate for each of the ten regression models considered 
- there is no consensus among the curves as to which model results in the smallest validation set MSE. 

Based on the variability among these curves, all that we can conclude with any confidence is that the linear fit is not adequate for this data. The validation set approach is conceptually simple and is easy to implement. But it has two potential drawbacks:

1. As is shown in the right-hand panel of Figure 5.2, the validation estimate of the test error rate can be highly variable, depending on precisely which observations are included in the training set and which observations are included in the validation set.
2. In the validation approach, only a subset of the observations—those that are included in the training set rather than in the validation set—are used to fit the model. Since statistical methods tend to perform worse when trained on fewer observations, this suggests that the validation set error rate may tend to overestimate the test error rate for the model fit on the entire data set.
   
In the coming subsections, we will present cross-validation, a refinement of the validation set approach that addresses these two issues.

## Leave-One-Out Cross-Validation

<!-- Leave-one-out cross-validation (LOOCV) is closely related to the validation set approach of Section 5.1.1, but it attempts to address that method’s drawbacks. -->

Leave-one-out cross-validation (LOOCV) is closely related to the validation set approach. It also splits the set of observations into two parts. However, instead of creating two subsets of comparable size, a single observation $(\mathbf{x}_i , y_i)$, where $ i = 1, 2, \dots, N $, is used for the validation set, and the remaining observations $\{(\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2), \dots, (\mathbf{x}_{i-1}, y_{i-1}), (\mathbf{x}_{i+1}, y_{i+1}), \dots, (\mathbf{x}_N , y_N)\}$ make up the training set. The statistical learning method is fit on the $ N − 1 $ training observations, and a prediction $\hat{y}_i$ is made for the excluded observation, using its value $\mathbf{x}_i$. 
Since $(\mathbf{x}_i , y_i)$ was not used in the fitting process, the squared test error $ \epsilon_i = (y_i − \hat{y}_i)^2 $ provides an approximately unbiased estimate. 
<!-- But even though MSE 1 is unbiased for the test error, it is a poor estimate because it is highly variable, since it is based upon a single observation (x 1 , y 1 ). -->

We can repeat the procedure by iterating $i$ from $1$ to $N$ to produce $N$ squared errors, $ \epsilon_i, \dots, \epsilon_N $. The LOOCV estimate for the test MSE is the average of these $N$ test error estimates:

$$\textrm{MSE} = \frac{1}{N}\sum_{i=1}^N \epsilon_i = \frac{1}{N}\sum_{i=1}^N (y_i − \hat{y}_i)^2 $$


Compared to the validation set approach, LOOCV has the following advantages:
- LOOCV has far less bias. In LOOCV, we repeatedly fit the statistical learning method using training sets that contain $N − 1$ observations, almost as many as are in the entire data set. This is in contrast to the validation set approach, in which the training set is typically around half the size of the original data set. Consequently, the LOOCV approach tends not to overestimate the test error rate as much as the validation set approach does. 
- In contrast to the validation approach which will yield different results when applied repeatedly due to randomness in the training/validation set splits, performing LOOCV multiple times will always yield the same results: there is no randomness in the training/validation set splits.


## K-Fold cross-validation

An alternative to LOOCV is k-fold CV. This approach involves randomly dividing the set of observations into k groups, or folds, of approximately equal size. Then repeating the following procedure by iterating $i$ from $1$ to $k$:

1. Treating $i\text{th}$ group of observations (fold) as a validation set,
2. Fitting model on the remaining $k − 1$ folds,
3. Computing the mean squared error, $\textrm{MSE}_i$

This process results in $k$ estimates of the test error, $\textrm{MSE}_1, \textrm{MSE}_2, \dots, \textrm{MSE}_k$ . The k-fold CV estimate is computed by averaging these values, 

LOOCV can be viewed as a special case of k-fold CV in which $k$ is set to equal $N$. In practice, one typically performs k-fold CV using $k = 5$ or $k = 10$. The most obvious advantage is computational. LOOCV requires fitting the statistical learning method $N$ times. This has the potential to be computationally expensive. But cross-validation is a very general approach that can be applied to almost any statistical learning method. Some statistical learning methods have computationally intensive fitting procedures, and so performing LOOCV may pose computational problems, especially if $N$ is extremely large. In contrast, performing 10-fold CV requires fitting the learning procedure only ten times, which may be much more feasible.

Run the following code to perform LOOCV and 10-fold CV on the `Auto` data:

In [ ]:
p_order = np.arange(1, 11)
r_state = np.arange(0, 10)

# LeaveOneOut CV
regr = LinearRegression()
loo = LeaveOneOut()
loo.get_n_splits(auto_df)
scores = list()

for i in p_order:
    poly = PolynomialFeatures(i)
    X_poly = poly.fit_transform(auto_df.horsepower.values.reshape(-1, 1))
    score = cross_val_score(
        regr, X_poly, auto_df.mpg, cv=loo, scoring="neg_mean_squared_error"
    ).mean()
    scores.append(score)

In [ ]:
# k-fold CV
folds = 10
elements = len(auto_df.index)

X, Y = np.meshgrid(p_order, r_state, indexing="ij")
Z = np.zeros((p_order.size, r_state.size))

regr = LinearRegression()

for (i, j), v in np.ndenumerate(Z):
    poly = PolynomialFeatures(X[i, j])
    X_poly = poly.fit_transform(auto_df.horsepower.values.reshape(-1, 1))
    kf_10 = KFold(n_splits=folds, random_state=Y[i, j], shuffle=True)
    Z[i, j] = cross_val_score(
        regr, X_poly, auto_df.mpg, cv=kf_10, scoring="neg_mean_squared_error"
    ).mean()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Note: cross_val_score() method return negative values for the scores.
# https://github.com/scikit-learn/scikit-learn/issues/2439

# Left plot
ax1.plot(p_order, np.array(scores) * -1, "-o")
ax1.set_title("LOOCV")

# Right plot
ax2.plot(X, Z * -1)
ax2.set_title("10-fold CV")

for ax in fig.axes:
    ax.set_ylabel("Mean Squared Error")
    ax.set_ylim(15, 30)
    ax.set_xlabel("Degree of Polynomial")
    ax.set_xlim(0.5, 10.5)
    ax.set_xticks(range(2, 11, 2));

The result of LOOCV and 10-fold CV are shown in the left and right panel, respectively. As we can see from the figure, there is some variability in the 10-fold CV estimates as a result of the variability in how the observations are divided into ten folds. But this variability is typically much lower than the variability in the test error estimates that results from the validation set approach.

The objective of performing cross-validation can be different:
- For model evaluation, our goal might be to determine how well a given statistical learning procedure can be expected to perform on independent data; in this case, the actual estimate of the test MSE is of interest. 
- For model selection, we are interested only in the location of the minimum point in the estimated test MSE curve. This is because we might be performing cross-validation on a number of statistical learning methods, or on a single method using different levels of flexibility, in order to identify the method that results in the lowest test error. For this purpose, the location of the minimum point in the estimated test MSE curve is important, but the actual value of the estimated test MSE is not. 

Read more about [cross-validation strategies in scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html)